In [ ]:
import PyPlot

In [ ]:
function MotionPrimitives(car_length,car_width,h,Δt,v_min,v_max,v_steps,δ_max,δ_min,δ_steps)
    """
    Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
    """
    # 3D array to store motion_primitives
    motion_primitives = zeros(v_steps,2*δ_steps+1,2) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,v_steps)*ones(1,2*δ_steps+1)
    δ = (linspace(-δ_max,δ_max,δ_steps*2+1)*ones(1,v_steps))' # steering angle

    motion_primitives[:,:,1] = v*Δt*h # arc length = velocity * time
    motion_primitives[:,:,2] = car_length./sin(δ) # radius of curvature (+ or -)
    motion_primitives[:,1+δ_steps,2] = Inf; # radius of curvature is infinite if steering angle = 0

    destination_primitives = zeros(v_steps,2*δ_steps+1,h,3) # lookup table defining car's location at each of next h time steps

    for i = 1:h
        # angle = 2π * arc_length / r
        dθ = v*Δt*i ./ motion_primitives[:,:,2]

        # dX = abs(radius) * sin(angle)
        destination_primitives[:,:,i,1] = abs(motion_primitives[:,:,2]) .* sin(abs(dθ))
        destination_primitives[:,1+δ_steps,i,1] = v[:,1+δ_steps]*Δt*i # centerline

        # dY = radius * (1 - cos(angle))
        destination_primitives[:,:,i,2] = motion_primitives[:,:,2].*(1 - cos(dθ))
        destination_primitives[:,1+δ_steps,i,2] = 0 # centerline

        destination_primitives[:,:,i,3] = dθ
    end

    # destination_primitives[v, δ, h, 1=Δx,2=Δy,3=Δθ]= changes in x, y and θ after h time steps
    
    return v, δ, destination_primitives
end

In [ ]:
car_length = 4.8
car_width = 1.5
h = 10
Δt = 1/24.0
v_min = 0.0
v_max = 100.0
δ_max = Float64(π)/6.0
δ_min = -δ_max


In [ ]:
δ_step = Float64(π)/128

ΔT = 2.5 # seconds
ΔV = 60*1600/3600 # 0->60 mph = 0->26.6 m/s
a_max = ΔV/ΔT # maximum acceleration = 10.66 m/s^2
v_step = a_max*Δt

In [ ]:
a_max

In [ ]:
"""
Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
"""
# 3D array to store motion_primitives
# radius = zeros(v_steps,2*δ_steps+1) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

v = linspace(v_min,v_max,round(v_max/v_step))*ones(1,2*Int(round((δ_max)/δ_step))+1) # velocity
δ = (linspace(-δ_max,δ_max,2*round((δ_max)/δ_step)+1)*ones(1,Int(round(v_max/v_step))))' # steering angle

radius = car_length./sin(δ) # radius of curvature (+ or -)
radius[:,Int(round((δ_max)/δ_step))+1] = Inf; # radius of curvature is infinite if steering angle = 0

destination_primitives = zeros(size(v,1),size(v,2),3,h,3)

accels = [-a_max,0,a_max]
for j in 1:3
    v₀ = copy(v)
    a = accels[j]
    for i = 1:h
        vᵢ = v₀+a*i*Δt # velocity at end of time step
        tire_force = sqrt((((vᵢ).^2)./abs(radius)).^2 + a^2) # tire force must be less than μ*g
        v_threshold = ones(size(v,1),1)*maximum(vᵢ.*(tire_force .< μ*g), 1)
        vᵢ = min(vᵢ,v_threshold)
        Δs = max((v₀+vᵢ)/2.0, 0.0)*i*Δt
        
        Δθ = Δs./radius # change in heading
        v₀[1:end-i,:] = v[i+1:end,:] # increase v₀ by one step for next round

        # dX = abs(radius) * sin(angle)
        destination_primitives[:,:,j,i,1] = abs(radius) .* sin(abs(Δθ))
        destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,1] = Δs[:,Int(round((δ_max)/δ_step))+1] # centerline

        # dY = radius * (1 - cos(angle))
        destination_primitives[:,:,j,i,2] = radius.*(1 - cos(Δθ))
        destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,2] = 0 # centerline

        destination_primitives[:,:,j,i,3] = Δθ
    end
end

In [ ]:
function MotionPrimitivesAcceleration(wheel_base,h,Δt,v_min,v_max,v_step,δ_min,δ_max,δ_step)
    """
    Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
    """
    # 3D array to store motion_primitives
    # radius = zeros(v_steps,2*δ_steps+1) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,round(v_max/v_step))*ones(1,2*Int(round((δ_max)/δ_step))+1) # velocity
    δ = (linspace(-δ_max,δ_max,2*round((δ_max)/δ_step)+1)*ones(1,Int(round(v_max/v_step))))' # steering angle

    radius = wheel_base./sin(δ) # radius of curvature (+ or -)
    radius[:,Int(round((δ_max)/δ_step))+1] = Inf; # radius of curvature is infinite if steering angle = 0

    destination_primitives = zeros(size(v,1),size(v,2),3,h,3)

    accels = [-a_max,0,a_max]
    for j in 1:3
        v₀ = copy(v)
        a = accels[j]
        for i = 1:h
            vᵢ = max(v₀+a*i*Δt, 0.0) # velocity at end of time step
            tire_force = sqrt((((vᵢ).^2)./abs(radius)).^2 + a^2) # tire force must be less than μ*g
            v_threshold = ones(size(v,1),1)*maximum(vᵢ.*(tire_force .< μ*g), 1)
            vᵢ = min(vᵢ,v_threshold)
            Δs = max((v₀+vᵢ)/2.0, 0.0)*i*Δt

            Δθ = Δs./radius # change in heading
            if j == 3 # accel = -1
                v₀[1:end-i,:] = v[i+1:end,:] # increase v₀ by one step for next round
            elseif j == 1 # accel = +1
                v₀[i+1:end,:] = v[1:end-i,:] # increase v₀ by one step for next round
            end
            # dX = abs(radius) * sin(angle)
            destination_primitives[:,:,j,i,1] = abs(radius) .* sin(abs(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,1] = Δs[:,Int(round((δ_max)/δ_step))+1] # centerline

            # dY = radius * (1 - cos(angle))
            destination_primitives[:,:,j,i,2] = radius.*(1 - cos(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,2] = 0 # centerline

            destination_primitives[:,:,j,i,3] = Δθ
        end
    end
    tire_force = sqrt((((v).^2)./abs(radius)).^2) # tire force must be less than μ*g
    mask = (tire_force .< μ*g)
    v_threshold = ones(size(v,1),1)*maximum(v.*mask, 1)
    v = min(v,v_threshold)
    
    return v, δ, destination_primitives, mask
end

In [ ]:
function MotionPrimitivesAcceleration(wheel_base,h,Δt,v_min,v_max,v_step,δ_min,δ_max,δ_step)
    """
    Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
    """
    # 3D array to store motion_primitives
    # radius = zeros(v_steps,2*δ_steps+1) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,round(v_max/v_step))*ones(1,2*Int(round((δ_max)/δ_step))+1) # velocity
    δ = (linspace(-δ_max,δ_max,2*round((δ_max)/δ_step)+1)*ones(1,Int(round(v_max/v_step))))' # steering angle

    radius = wheel_base./sin(δ) # radius of curvature (+ or -)
    radius[:,Int(round((δ_max)/δ_step))+1] = Inf; # radius of curvature is infinite if steering angle = 0
    
    n_actions = 3 # slow down, stay, speed up
    destination_primitives = zeros(size(v,1),size(v,2),n_actions,h,3)

    accels = [-a_max,0,a_max]
    for j in 1:3
        v₀ = copy(v)
        a = accels[j]
        for i = 1:h
            vᵢ = max(v₀+a*i*Δt, 0.0) # velocity at end of time step
            tire_force = sqrt((((vᵢ).^2)./abs(radius)).^2 + a^2) # tire force must be less than μ*g
            v_threshold = ones(size(v,1),1)*maximum(vᵢ.*(tire_force .< μ*g), 1)
            vᵢ = min(vᵢ,v_threshold)
            Δs = max((v₀+vᵢ)/2.0, 0.0)*i*Δt

            Δθ = Δs./radius # change in heading
            if j == 3 # accel = -1
                v₀[1:end-i,:] = v[i+1:end,:] # increase v₀ by one step for next round
            elseif j == 1 # accel = +1
                v₀[i+1:end,:] = v[1:end-i,:] # increase v₀ by one step for next round
            end
            # dX = abs(radius) * sin(angle)
            destination_primitives[:,:,j,i,1] = abs(radius) .* sin(abs(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,1] = Δs[:,Int(round((δ_max)/δ_step))+1] # centerline

            # dY = radius * (1 - cos(angle))
            destination_primitives[:,:,j,i,2] = radius.*(1 - cos(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,2] = 0 # centerline

            destination_primitives[:,:,j,i,3] = Δθ
        end
    end
    
    tire_force = sqrt((((v).^2)./abs(radius)).^2) # tire force must be less than μ*g
    mask = (tire_force .< μ*g)
    v_threshold = ones(size(v,1),1)*maximum(v.*mask, 1)
    v = min(v,v_threshold)
    motion_map = Dict()
    for i in 1:size(v,1) # motion_map[i] contains the reachable states from v[i]
        motion_map[i] = destination_primitives[i,:,:,:,:]
    end
    
    return v, δ, destination_primitives, motion_map, mask
end

In [ ]:
v, δ, destination_primitives, motion_map, mask = MotionPrimitivesAcceleration(car_length,h,Δt,v_min,v_max,v_step,δ_min,δ_max,δ_step)
destination_primitives;

In [ ]:
v_idx = 3
δ_idx = 21
a_idx = 1
ΔXYθ = motion_map[v_idx][:,:,10,:]
trajectory = motion_map[v_idx][δ_idx,a_idx,:,:]
PyPlot.scatter(ΔXYθ[:,:,2],ΔXYθ[:,:,1],edgecolor="none")
PyPlot.scatter(trajectory[:,2],trajectory[:,1],c="red")
PyPlot.axis("equal")

In [ ]:
radius = car_length./sin(δ) # radius of curvature (+ or -)
# mask for tire saturation
tire_force⁺ = sqrt(((v₀.^2)./abs(radius)).^2 + a_max^2)
tire_force⁰ = sqrt(((v₀.^2)./abs(radius)).^2 + 0)
tire_force⁻ = sqrt(((v₀.^2)./abs(radius)).^2 + a_max^2)

# Plot tire force saturation threshold
PyPlot.figure(figsize=[12,4])
PyPlot.subplot(1,3,1)
PyPlot.title("Tire Forces +")
PyPlot.scatter(δ[tire_force⁺ .< μ*g]*180.0/Float64(π),v[tire_force⁺ .< μ*g],c=tire_force⁺[tire_force⁺ .< μ*g],edgecolor="none")
PyPlot.xlabel("Steering Angle (degrees)")
PyPlot.ylabel("Tangential Velocity (m/s)")
PyPlot.axis("tight")
PyPlot.subplot(1,3,2)
PyPlot.title("Tire Forces 0")
PyPlot.scatter(δ[tire_force⁰ .< μ*g]*180.0/Float64(π),v[tire_force⁰ .< μ*g],c=tire_force⁰[tire_force⁰ .< μ*g],edgecolor="none")
PyPlot.xlabel("Steering Angle (degrees)")
# PyPlot.ylabel("Tangential Velocity (m/s)")
PyPlot.axis("tight")
PyPlot.subplot(1,3,3)
PyPlot.title("Tire Forces -")
PyPlot.scatter(δ[tire_force⁻ .< μ*g]*180.0/Float64(π),v[tire_force⁻ .< μ*g],c=tire_force⁻[tire_force⁻ .< μ*g],edgecolor="none")
PyPlot.xlabel("Steering Angle (degrees)")
# PyPlot.ylabel("Tangential Velocity (m/s)")
PyPlot.axis("tight")

In [ ]:
PyPlot.scatter(δ[tire_force⁰ .< μ*g],v[tire_force⁰ .< μ*g],c=tire_force⁰[tire_force⁰ .< μ*g],edgecolor="none")

In [ ]:
# v_threshold = ones(size(v,1),1)*maximum(v₀.*(tire_force⁰ .< μ*g),1)
# v₀[1:end-1,:] = v[1+1:end,:]
# v₀ = min(v₀,v_threshold)
PyPlot.scatter(δ,v,edgecolor="none",c=v)

## Modify HRHC parameters

In [1]:
using AutomotiveDrivingModels
using AutoViz
using SplineRaceWay
using SplineUtils
using NearestNeighbors

/Users/kyle/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
function CurveDist(pt1::CurvePt, pt2::CurvePt)
    d = sqrt((pt1.pos.x - pt2.pos.x)^2 + (pt1.pos.y - pt2.pos.y)^2)
end
function wrap_to_π(θ)
    θ = θ - div(θ,2*Float64(π))*(2*Float64(π))
    θ = θ + (θ .< -π).*(2*Float64(π)) - (θ .> π).*(2*Float64(π))
end

wrap_to_π (generic function with 1 method)

In [3]:
type accelHRHC <: DriverModel{NextState, IntegratedContinuous}
    action_context::IntegratedContinuous
    v_cmds # possible velocity commands
    δ_cmds # possible δ_cmds
    ΔXYθ # state changes associated with cmd = (v_command, δ_command) - this is the motion map
    eligibility_mask # logical array with zeros at locations of tire force saturation
    v_idx::Int # index of v in v_cmds
    δ_idx::Int # index of δ in δ_cmds
    successor_states # reachable next_states

    #car parameters with bicycle geometry model
    car_length::Float64 # wheel base
    car_width::Float64
    car_ID::Int

    # current v, current δ
    v::Float64
    δ::Float64
    curve_ind::Int
    Δs::Float64

    # planning horizon
    h::Int
    Δt::Float64

    # logit level
    k::Int

    V_MIN::Float64
    V_MAX::Float64
    
    a_max::Float64 # max acceleration (m/s)
    μ::Float64 # friction coefficient

    # maximum deviation from center of track (if |t| > T_MAX, car is out of bounds)
    T_MAX::Float64

    # Action = Next State
    action::NextState

    function accelHRHC(car_ID::Int,roadway,context;
        car_length::Float64=4.8,
        car_width::Float64=2.5,
        v::Float64=0.0,
        δ::Float64=0.0,
        h::Int=10,
        Δt::Float64=1.0/24,
        ΔV₊::Float64=1.55,
        ΔV₋::Float64=3.05,
        Δδ::Float64=Float64(π)/12,
        V_MIN::Float64=0.0,
        V_MAX::Float64=100.0,
        a_max::Float64=10.0,
        μ::Float64=20.0,
        g::Float64=9.81,
        δ_MAX::Float64=Float64(π)/8,
        δ_MIN::Float64=-Float64(π)/8,
        δ_STEP::Float64=Float64(π)/64,
        k::Int=1
        )

        hrhc = new()

        hrhc.V_MIN=V_MIN
        hrhc.V_MAX=V_MAX
        
        hrhc.T_MAX=(roadway.segments[1].lanes[1].width - car_width)/2.0

        hrhc.car_ID = car_ID
        hrhc.car_length=car_length
        hrhc.car_width=car_width
        hrhc.h=h
        hrhc.Δt=Δt
        hrhc.a_max=a_max
        hrhc.μ=μ

        hrhc.k=k
        v_step = a_max*Δt

        hrhc.v_cmds, hrhc.δ_cmds, _, hrhc.ΔXYθ, hrhc.eligibility_mask  = MotionPrimitivesAcceleration(car_length,h,Δt,V_MIN,V_MAX,a_max,v_step,δ_MIN,δ_MAX,δ_STEP,μ,g)

        hrhc.v=v
        hrhc.δ=δ
        hrhc.v_idx = 1
        hrhc.δ_idx = Int((size(hrhc.δ_cmds, 2) - 1)/2)
        hrhc.curve_ind=1
        hrhc.Δs=roadway.segments[1].lanes[1].curve[2].s-roadway.segments[1].lanes[1].curve[1].s
        hrhc.action_context=context
        hrhc.action = NextState(VehicleState(VecSE2(0,0,0),0.0))

        hrhc
    end
end

In [4]:
function generateObstacleMap(scene, models)
    k = maximum([driver.k for (id, driver) in models])
    n = length(scene)
    h = maximum([driver.h for (id, driver) in models]) # h should be the same for all vehicles on the track
    obstacleDict = Dict()
    for level in 0:k
        obstacleDict[level] = Dict()
        for (id, driver) in models
            obstacleDict[level][id] = zeros(h,5) # x,y,θ,v,δ
        end
    end

    return obstacleDict
end
function updateObstacleMap!(obstacleMap, level, car_ID, trajectory)
    obstacleMap[level][car_ID][1:size(trajectory,1),1:size(trajectory,2)] = trajectory
end
function getObstacleCoords(obstacleMap, level, car_ID, h)
    return obstacleMap[level][car_ID][h,:]
end

getObstacleCoords (generic function with 1 method)

In [5]:
function MotionPrimitivesAcceleration(wheel_base,h,Δt,v_min,v_max,a_max,v_step,δ_min,δ_max,δ_step,μ,g)
    """
    Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
    """
    # 3D array to store motion_primitives
    # radius = zeros(v_steps,2*δ_steps+1) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,round(v_max/v_step))*ones(1,2*Int(round((δ_max)/δ_step))+1) # velocity
    δ = (linspace(-δ_max,δ_max,2*round((δ_max)/δ_step)+1)*ones(1,Int(round(v_max/v_step))))' # steering angle

    radius = wheel_base./sin(δ) # radius of curvature (+ or -)
    radius[:,Int(round((δ_max)/δ_step))+1] = Inf; # radius of curvature is infinite if steering angle = 0
    
    n_actions = 3 # slow down, stay, speed up
    destination_primitives = zeros(size(v,1),size(v,2),n_actions,h,3)

    accels = [-a_max,0,a_max]
    for j in 1:3
        v₀ = copy(v)
        a = accels[j]
        for i = 1:h
            vᵢ = max(v₀+a*i*Δt, 0.0) # velocity at end of time step
            tire_force = sqrt((((vᵢ).^2)./abs(radius)).^2 + a^2) # tire force must be less than μ*g
            v_threshold = ones(size(v,1),1)*maximum(vᵢ.*(tire_force .< μ*g), 1)
            vᵢ = min(vᵢ,v_threshold)
            Δs = max((v₀+vᵢ)/2.0, 0.0)*i*Δt

            Δθ = Δs./radius # change in heading
            if j == 3 # accel = -1
                v₀[1:end-i,:] = v[i+1:end,:] # increase v₀ by one step for next round
            elseif j == 1 # accel = +1
                v₀[i+1:end,:] = v[1:end-i,:] # increase v₀ by one step for next round
            end
            # dX = abs(radius) * sin(angle)
            destination_primitives[:,:,j,i,1] = abs(radius) .* sin(abs(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,1] = Δs[:,Int(round((δ_max)/δ_step))+1] # centerline

            # dY = radius * (1 - cos(angle))
            destination_primitives[:,:,j,i,2] = radius.*(1 - cos(Δθ))
            destination_primitives[:,Int(round((δ_max)/δ_step))+1,j,i,2] = 0 # centerline

            destination_primitives[:,:,j,i,3] = Δθ
        end
    end
    
    tire_force = sqrt((((v).^2)./abs(radius)).^2) # tire force must be less than μ*g
    mask = (tire_force .< μ*g)
    v_threshold = ones(size(v,1),1)*maximum(v.*mask, 1)
    v = min(v,v_threshold)
    motion_map = Dict()
    for i in 1:size(v,1) # motion_map[i] contains the reachable states from v[i]
        motion_map[i] = destination_primitives[i,:,:,:,:]
    end
    
    return v, δ, destination_primitives, motion_map, mask
end

MotionPrimitivesAcceleration (generic function with 1 method)

In [6]:
function getSuccessorStates!(hrhc::accelHRHC, scene::Scene)
    """ gets legal successor_states from motion primitives library """
    pos = scene.vehicles[hrhc.car_ID].state.posG # global x,y,z of car

    ΔX = hrhc.ΔXYθ[hrhc.v_idx][:,:,:,1] * cos(pos.θ) + hrhc.ΔXYθ[hrhc.v_idx][:,:,:,2] * -sin(pos.θ)
    ΔY = hrhc.ΔXYθ[hrhc.v_idx][:,:,:,1] * sin(pos.θ) + hrhc.ΔXYθ[hrhc.v_idx][:,:,:,2] * cos(pos.θ)
    Δθ = hrhc.ΔXYθ[hrhc.v_idx][:,:,:,3]

    hrhc.successor_states = zeros(size(hrhc.ΔXYθ[hrhc.v_idx]))
    hrhc.successor_states[:,:,:,1] = ΔX + pos.x
    hrhc.successor_states[:,:,:,2] = ΔY + pos.y
    hrhc.successor_states[:,:,:,3] = Δθ + pos.θ

    return
end

getSuccessorStates! (generic function with 1 method)

In [7]:
function computeTrajectory(hrhc::accelHRHC, scene, cmd_index; h=hrhc.h)
    pos = scene.vehicles[hrhc.car_ID].state.posG

    traj_ΔXYθ = hrhc.ΔXYθ[hrhc.v_idx][cmd_index[2],cmd_index[1],1:h,:]

    ΔX = traj_ΔXYθ[:,1] * cos(pos.θ) + traj_ΔXYθ[:,2] * -sin(pos.θ)
    ΔY = traj_ΔXYθ[:,1] * sin(pos.θ) + traj_ΔXYθ[:,2] * cos(pos.θ)
    Δθ = traj_ΔXYθ[:,3]

    trajectory = zeros(size(traj_ΔXYθ,1),size(traj_ΔXYθ,2)+2)
    trajectory[:,1] = ΔX + pos.x
    trajectory[:,2] = ΔY + pos.y
    trajectory[:,3] = Δθ + pos.θ
    trajectory[:,4] = hrhc.v_cmds[cmd_index[1],cmd_index[2]]
    trajectory[:,5] = hrhc.δ_cmds[cmd_index[1],cmd_index[2]]

    return trajectory
end

computeTrajectory (generic function with 1 method)

In [8]:
function loopProjectionKD(hrhc::accelHRHC,scene,roadway,tree)
    """
    projects all points in hrhc.successor_states to the kdtree representing
    the spline points along the centerline of roadway
    """
    curve = roadway.segments[1].lanes[1].curve

    s_grid = zeros(size(hrhc.successor_states,1),size(hrhc.successor_states,2))
    t_grid = zeros(size(s_grid))
    ϕ_grid = zeros(size(s_grid))
    idx_grid = zeros(Int,size(s_grid))

    pts = [reshape(hrhc.successor_states[:,:,1],size(hrhc.successor_states[:,:,1],1)*size(hrhc.successor_states[:,:,1],2),1)';
    reshape(hrhc.successor_states[:,:,2],size(hrhc.successor_states[:,:,2],1)*size(hrhc.successor_states[:,:,2],2),1)']
    idxs_list, _ = knn(tree,pts,1)
    idxs=reshape(idxs_list,size(hrhc.successor_states[:,:,2],1),size(hrhc.successor_states[:,:,2],2))


    for i in 1:size(s_grid,1)
        for j in 1:size(s_grid,2)
            idxA = idxs[i,j][1]-1
            idxB = idxs[i,j][1]+1
            if idxs[i,j][1] == length(curve)
                idxB = 1 # wrap to the beginning of the curve
            end
            if idxs[i,j][1] == 1
                idxA = length(curve) # wrap to the end of the curve
            end
            x = hrhc.successor_states[i,j,1]
            y = hrhc.successor_states[i,j,2]
            dA = sqrt(sum(([curve[idxA].pos.x, curve[idxA].pos.y]-[x,y]).^2))
            dB = sqrt(sum(([curve[idxB].pos.x, curve[idxB].pos.y]-[x,y]).^2))
            if dA < dB
                idxB = idxs[i,j][1]
            else
                idxA = idxs[i,j][1]
            end

            # project
            vec1 = [curve[idxB].pos.x - curve[idxA].pos.x, curve[idxB].pos.y - curve[idxA].pos.y, 0]
            vec2 = [x - curve[idxA].pos.x, y - curve[idxA].pos.y, 0]
            idx_t = dot(vec2, vec1)/norm(vec1)^2

            s_θ = curve[idxA].pos.θ + idx_t*(curve[idxB].pos.θ - curve[idxA].pos.θ)

            s_grid[i,j] = curve[idxA].s + idx_t*hrhc.Δs
            t_grid[i,j] = norm(vec2 - idx_t*vec1)*sign(sum(cross(vec1, vec2)))
            ϕ_grid[i,j] = wrap_to_π(hrhc.successor_states[i,j,3] - s_θ)
            idx_grid[i,j] = idxA
        end
    end
    # account for wrap-around
    s_grid[s_grid .< scene.vehicles[hrhc.car_ID].state.posF.s] += curve[end].s + hrhc.Δs

    return s_grid, t_grid, ϕ_grid
end

loopProjectionKD (generic function with 1 method)

In [9]:
function calculateObjective(hrhc::accelHRHC, scene, roadway, tree, s, t, ϕ, obstacleMap, k_level, h; f_ϕ=0.0, f_t=0.1, f_tϕ=3.0)
    """
    Calculates the value of the optimization objective function for every state
      in hrhc.successor_states
    """
    state = scene.vehicles[hrhc.car_ID].state
    dS = s - state.posF.s
    dS = dS / maximum(dS) # normalize
    ϕMAX = Float64(π)/2

    # penalize large t (proximity to edge of track)
    cost_t = (exp(((10-h+f_t)*abs(t/hrhc.T_MAX).^2)) - 1)/exp(f_t) + Inf*(t.>hrhc.T_MAX)
    # penalize large ϕ (steering away from forward direction on the track)
    cost_ϕ = (exp(((10-h+f_ϕ)*abs(ϕ/ϕMAX).^2)) - 1)/exp(f_ϕ)
    # penalize when t and ϕ have the same sign
    A = [1 1; 1 1]
    cost_x = (((ϕ/ϕMAX)*A[1,1] + (t/hrhc.T_MAX)*A[2,1]).*(ϕ/ϕMAX) + ((ϕ/ϕMAX)*A[1,2] + (t/hrhc.T_MAX)*A[2,2]).*(t/hrhc.T_MAX))/2
    cost_tϕ = (exp(f_tϕ*cost_x) - 1)/exp(1)
    eligibility_mask = ((hrhc.successor_states[:,:,1] .== state.posG.x).*(hrhc.successor_states[:,:,2] .== state.posG.y))

    # obstacles
    collisionCost = zeros(size(cost_t))
    threshold_dist = hrhc.car_length*4 # must be at least this close before we care to calculate collision cost
    if k_level >= 1
        for (id,car) in obstacleMap[k_level - 1]
            if id != hrhc.car_ID
                state = scene.vehicles[hrhc.car_ID].state
                state2 = scene.vehicles[id].state
                diff = state.posG - state2.posG
                s1,_,_ = kdProject(state.posG.x,state.posG.y,state.posG.θ,tree,roadway,hrhc)
                s2,_,_ = kdProject(state2.posG.x,state2.posG.y,state2.posG.θ,tree,roadway,hrhc)
                if (norm([diff.x, diff.y]) < threshold_dist) && (s1 <= s2) # don't care if opponent is behind us
                    pos = VecSE2(car[h,1:3]) # x,y,θ of opponent at time step h
                    ΔX = hrhc.successor_states[:,:,1] - pos.x # Δx, with opponent at origin
                    ΔY = hrhc.successor_states[:,:,2] - pos.y # Δy with opponent at origin
                    Δθ = hrhc.successor_states[:,:,3] - pos.θ # Δθ with opponent at origin
                    pts = [hrhc.car_length hrhc.car_length -hrhc.car_length -hrhc.car_length 0;
                        -hrhc.car_width hrhc.car_width hrhc.car_width -hrhc.car_width 0]/1.8
                    pX = zeros(size(pts,2),size(hrhc.successor_states,1),size(hrhc.successor_states,2))
                    pY = zeros(size(pX))
                    for i in 1:size(pts,2)
                        pX[i,:,:] = pts[1,i]*cos(Δθ) - pts[2,i]*sin(Δθ) + ΔX
                        pY[i,:,:] = pts[1,i]*sin(Δθ) + pts[2,i].*cos(Δθ) + ΔY
                    end

                    collisionFlag = (maximum((abs(pX) .< hrhc.car_length/1.0),1)[1,:,:]).*(maximum((abs(pY) .< hrhc.car_width/1.9),1)[1,:,:])
                    collisionCost = .001+(collisionFlag .>= 1)./(minimum(abs(pX),1)[1,:,:].*minimum(abs(pY),1)[1,:,:])
                    # collisionCost = Inf.*collisionFlag
                end
            end
        end
    end

    objective = cost_t + cost_ϕ + cost_tϕ + 1 - dS + collisionCost + Inf * eligibility_mask
    return objective
end

calculateObjective (generic function with 1 method)

In [10]:
# scene parameters
scene = Scene()
framerate = 24
Δt = 1.0/framerate # size of rendering timesteps
n_integration_sub_steps = 3 # sub steps for smoother integration
context = IntegratedContinuous(Δt, n_integration_sub_steps) # integrated continuous context

#car parameters
car_length = 4.8 # front wheel to back wheel
car_width = 2.5
v⁰  = 0.0 # initial velocity
δ⁰ = 0.0 # initical steering angle

############### INITIALIZE TRACK ################

lane_width = 8.0*DEFAULT_LANE_WIDTH
radius = 45.0
edge_buffer = 0.25
T_MAX = lane_width/2.0 - car_width/2.0 - edge_buffer # max allowable projection distance from center of lane
base_speed= 0.0

# spline control points
# Pts = 40*[0 -1 -2 -3 -3 -3 -2 -1 -1 -1 -2 -3 -4 -5 -5 -5 -5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
#          0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]
Pts = 40*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -5.5 -5 -4.5 -5 -5 -4 -3 -2 -1 -1 -1 0 1 1 1 2 3 4 5 5 5 5 5 5 5 4 3 3 3 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -3 -2 -1 0 0 0 0]

degree = 3 # degree of spline
num_points = 10001
num_samples = 420
lane_width = 20.0
track = Raceway(Pts,degree,num_points,num_samples,lane_width)
track;

In [11]:
car = accelHRHC(1,track.roadway,context)
car;

In [12]:
roadind1 = RoadIndex(CurveIndex(20,0),LaneTag(1,1))
vehstate1 = VehicleState(Frenet(roadind1, track.roadway), track.roadway, 0.0)
vehdef1 = VehicleDef(1,AgentClass.CAR, car_length, car_width)
push!(scene,Vehicle(vehstate1, vehdef1))

carcolors = Dict{Int, Colorant}()
carcolors[1] = colorant"red"

# hrhc = HRHC(1,roadway,context)
track.models[1] = car
track.obstacleMap = generateObstacleMap(scene, track.models)
actions = Array(DriveAction, length(scene))

1-element Array{AutomotiveDrivingModels.DriveAction,1}:
 #undef

In [13]:
size(car.ΔXYθ[car.v_idx])

(17,3,10,3)

In [14]:
getSuccessorStates!(car,scene)
action = 3
car.successor_states[:,action,car.h,:]

17×3 Array{Float64,2}:
 -85.7005   0.196803    -3.32661
 -85.7037   0.168677    -3.30336
 -85.7065   0.139998    -3.2797 
 -85.7089   0.110843    -3.25568
 -85.7108   0.0812864   -3.23137
 -85.7122   0.0514086   -3.20681
 -85.7132   0.0212893   -3.18208
 -85.7137  -0.00898994  -3.15723
 -85.7136  -0.0393471   -3.13232
 -85.7131  -0.0696995   -3.1074 
 -85.7121  -0.0999648   -3.08255
 -85.7105  -0.130061    -3.05782
 -85.7085  -0.159907    -3.03326
 -85.7061  -0.189422    -3.00895
 -85.7032  -0.218529    -2.98493
 -85.6998  -0.247151    -2.96127
 -85.6961  -0.275213    -2.93802

In [16]:
s,t,ϕ = loopProjectionKD(car, scene, track.roadway, track.tree)

(
[229.795 229.795 229.787; 229.795 229.795 229.787; … ; 229.795 229.795 229.787; 229.795 229.795 229.787],

[-3.28483 -3.28483 -3.35445; -3.28483 -3.28483 -3.35445; … ; -3.28483 -3.28483 -3.35445; -3.28483 -3.28483 -3.35445],

[-1.59041 -1.59041 -1.77312; -1.59041 -1.59041 -1.77312; … ; -1.59041 -1.59041 -1.7731; -1.59041 -1.59041 -1.77309])

In [17]:
calculateObjective(car, scene, track.roadway, track.tree, s, t, ϕ, track.obstacleMap, car.k, car.h)

17×3 Array{Float64,2}:
 6.26021  6.26021  11.0042
 6.26021  6.26021  11.0042
 6.26021  6.26021  11.0042
 6.26021  6.26021  11.0042
 6.26021  6.26021  11.0041
 6.26021  6.26021  11.0041
 6.26021  6.26021  11.0041
 6.26021  6.26021  11.004 
 6.26021  6.26021  11.004 
 6.26021  6.26021  11.0039
 6.26021  6.26021  11.0039
 6.26021  6.26021  11.0038
 6.26021  6.26021  11.0037
 6.26021  6.26021  11.0036
 6.26021  6.26021  11.0035
 6.26021  6.26021  11.0034
 6.26021  6.26021  11.0033

In [15]:
trajectory = computeTrajectory(car,scene,(3,8))

10×5 Array{Float64,2}:
 -83.2853  -0.0168176   -3.1324   0.83682  -0.0490874
 -83.3462  -0.0173583   -3.13303  0.83682  -0.0490874
 -83.4594  -0.018262    -3.13418  0.83682  -0.0490874
 -83.6247  -0.0193475   -3.13587  0.83682  -0.0490874
 -83.8423  -0.0203499   -3.1381   0.83682  -0.0490874
 -84.1122  -0.0209208   -3.14086  0.83682  -0.0490874
 -84.4342  -0.0206278   -3.14415  0.83682  -0.0490874
 -84.8085  -0.0189553   -3.14797  0.83682  -0.0490874
 -85.235   -0.0153037   -3.15233  0.83682  -0.0490874
 -85.7137  -0.00898994  -3.15723  0.83682  -0.0490874

In [20]:
function screenTrajectory(trajectory, obstacleMap, scene, roadway, hrhc, tree, k_level)
    out_of_bounds = false
    collision_flag = false
    # check out of bounds
    for i in 1 : size(trajectory,1)
        x = trajectory[i,1]
        y = trajectory[i,2]
        θ = trajectory[i,3]
        s,t,ϕ = kdProject(x,y,θ,tree,roadway,hrhc)

        if abs(t) > hrhc.T_MAX
            out_of_bounds=true
            return out_of_bounds
        end
    end
    # check for collision
    # threshold_dist = 4.0*hrhc.car_length
    # if k_level >= 1
    #     for (id,car) in obstacleMap[k_level - 1]
    #         if id != hrhc.car_ID
    #             state = scene.vehicles[hrhc.car_ID].state
    #             state2 = scene.vehicles[id].state
    #             diff = state.posG - state2.posG
    #             s1,_,_ = kdProject(state.posG.x,state.posG.y,state.posG.θ,tree,roadway,hrhc)
    #             s2,_,_ = kdProject(state2.posG.x,state2.posG.y,state2.posG.θ,tree,roadway,hrhc)
    #             if (norm([diff.x, diff.y]) < threshold_dist) && (s1 <= s2) # check which car is ahead
    #                 for i in 1: size(trajectory,1)
    #                     egoCar = VecSE2(trajectory[i,1],trajectory[i,2],trajectory[i,3])
    #                     car2 = VecSE2(car[i,1],car[i,2],car[i,3])
    #                     collision_flag = checkCollision(scene,egoCar,car2,hrhc.car_ID,id)
    #                     if collision_flag
    #                         return collision_flag
    #                     end
    #                 end
    #             end
    #         end
    #     end
    # end

    return out_of_bounds
end

screenTrajectory (generic function with 1 method)

In [22]:
function kdProject(x,y,θ,tree,roadway,hrhc)
    """
    project single (x,y,Θ) point to roadway spline using kdtree to find the nearest spline point
    """
    curve = roadway.segments[1].lanes[1].curve
    # Δs = roadway.segments[1].lanes[1].curve[2].s - roadway.segments[1].lanes[1].curve[1].s
    idx_list,dist = knn(tree,[x;y],1)
    idx = idx_list[1]
    idxA = idx-1
    idxB = idx+1

    if idx == length(curve)
        idxB = 1 # back to the beginning of the curve
    end
    if idx == 1
        idxA = length(curve)
    end
    dA = sqrt(sum(([curve[idxA].pos.x, curve[idxA].pos.y]-[x,y]).^2))
    dB = sqrt(sum(([curve[idxB].pos.x, curve[idxB].pos.y]-[x,y]).^2))
    if dA < dB
        idxB = idx
    else
        idxA = idx
    end

    # project
    vec1 = [curve[idxB].pos.x - curve[idxA].pos.x, curve[idxB].pos.y - curve[idxA].pos.y, 0]
    vec2 = [x - curve[idxA].pos.x, y - curve[idxA].pos.y, 0]
    idx_t = dot(vec2, vec1)/norm(vec1)^2

    pθ = curve[idxA].pos.θ + idx_t*(curve[idxB].pos.θ - curve[idxA].pos.θ)

    s = curve[idxA].s + idx_t*hrhc.Δs
    t = -norm(vec2 - idx_t*vec1)
    ϕ = wrap_to_π(θ - pθ)

    s,t,ϕ,idxA
end

kdProject (generic function with 1 method)

In [25]:
observe!(car,scene,track.roadway,1,track.tree,track.obstacleMap,car.k)

NextState(VehicleState(VecSE2({-83.277, -0.017}, -3.132), Frenet(RoadIndex({20, -0.000000}, {1, 1}), 83.277, 0.000, 0.000), 0.000))

In [24]:
function AutomotiveDrivingModels.observe!(hrhc::accelHRHC, scene::Scene, roadway::Roadway, egoid::Int, tree::KDTree, obstacleMap, k_level)
    """
    Observe the current environment and select optimal action to apply at next
    time step
    """
    if k_level > hrhc.k
        return
    end
    state = scene.vehicles[hrhc.car_ID].state
    hrhc.curve_ind = state.posF.roadind.ind.i
    v = state.v # current v
    hrhc.v = v

    trajectory = zeros(hrhc.h,5)
    action_selected = false
    abs_cmd = (hrhc.v_idx,hrhc.δ_idx)
    v_cmd = hrhc.v_idx
    δ_cmd = hrhc.δ_idx

    i = 0
    for i in 0:(hrhc.h-1)
        if action_selected
            break # out of for loop
        end
        
        # calculate successor states
        getSuccessorStates!(hrhc, scene)

        # project successor states onto track
        s,t,ϕ = loopProjectionKD(hrhc, scene, roadway, tree)

        # optimization objective
        objective = calculateObjective(hrhc, scene, roadway, tree, s, t, ϕ, obstacleMap, k_level, hrhc.h-i,f_t=0.0)

        while (action_selected==false) && (minimum(objective) != Inf)
            index = indmin(objective) # find get a better method of optimizing this
            v_cmd,δ_cmd = ind2sub(s, index)

            # compute full trajectory up to horizon
            trajectory = computeTrajectory(hrhc, scene, (v_cmd,δ_cmd), h=hrhc.h-i)

            # screen trajectory for collisions / validity
            out_of_bounds = screenTrajectory(trajectory, obstacleMap, scene, roadway, hrhc, tree, k_level)

            if out_of_bounds
                objective[index] = Inf
            else
                action_selected=true
                updateObstacleMap!(obstacleMap, k_level, hrhc.car_ID, trajectory)
            end
        end
    end

    hrhc.δ_idx = δ_cmd
    hrhc.v_idx = min(max(1,hrhc.v_idx+v_cmd-2), size(hrhc.v,1))
    
    hrhc.δ = hrhc.δ_cmds[hrhc.v_idx, δ_cmd] # next δ
    hrhc.v = hrhc.v_cmds[hrhc.v_idx, δ_cmd] # next v

    next_state = VehicleState(VecSE2(trajectory[1,1:3]),roadway,hrhc.v)
    hrhc.action = NextState(next_state) # action
end